### Case Study project for NoSQL

#### Import necessary libraries to work with Mongodb

In [1]:
import pymongo
import json
from pprint import pprint 
from datetime import datetime

#### Connect to local mongodb and output the connection status

In [2]:
client = pymongo.MongoClient("localhost", 27017)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

#### Import a sample_analytics collection into local db (this is a one time activities)

In [3]:
#!"C:\Program Files\MongoDB\Tools\100\bin\mongoimport" --host localhost --port 27017 --db financial_analytics --collection accounts --file "./dataset/accounts.json"
#!"C:\Program Files\MongoDB\Tools\100\bin\mongoimport" --host localhost --port 27017 --db financial_analytics --collection customers --file "./dataset/customers.json"
#!"C:\Program Files\MongoDB\Tools\100\bin\mongoimport" --host localhost --port 27017 --db financial_analytics --collection transactions --file "./dataset/transactions.json"

#### Create a database financial_analytics and add three collection to the same, this are
- accounts
- customers
- transactions

In [4]:
finance_db = client["financial_analytics"]

    
accounts = finance_db["accounts"]
customers = finance_db["customers"]
transactions = finance_db["transactions"]

databases = client.list_database_names()
for dbs in databases:
    print(dbs)
    
print('========  financial_analytics Collections  ===========')
print(f"accounts count: {accounts.count_documents({})}")
print(f"customers count: {customers.count_documents({})}")
print(f"transactions count: {transactions.count_documents({})}")

admin
config
financial_analytics
local
========  financial_analytics Collections  ===========
accounts count: 1746
customers count: 500
transactions count: 1746


### Create index on following field to improve the search query

- Customers (Compound index on name and username)
- accounts (single index on limit)
- transactions (array index)


In [5]:
#create compound  index on customers
customers.create_index([("name",1),("username",1)]);

#create single field index on accounts
accounts.create_index([("limit",1)])

transactions.create_index([("transactions",1)])
#customers.create_index([("name", "text")])

'transactions_1'

### View default and custom index on the three collections

In [6]:
# Print the list of indexes
[print(index) for index in accounts.list_indexes()]
[print(index) for index in customers.list_indexes()]
[print(index) for index in transactions.list_indexes()]

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('limit', 1)])), ('name', 'limit_1')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('_fts', 'text'), ('_ftsx', 1)])), ('name', 'name_text'), ('weights', SON([('name', 1)])), ('default_language', 'english'), ('language_override', 'language'), ('textIndexVersion', 3)])
SON([('v', 2), ('key', SON([('name', 1), ('username', 1)])), ('name', 'name_1_username_1')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('transactions', 1)])), ('name', 'transactions_1')])


[None, None]

### Display one record from accounts collection

In [7]:
[pprint(records) for records in finance_db.accounts.find().limit(1)]

{'_id': ObjectId('5ca4bbc7a2dd94ee5816238d'),
 'account_id': 557378,
 'limit': 10000,
 'products': ['InvestmentStock', 'Commodity', 'Brokerage', 'CurrencyService']}


[None]

### Display one record from customers collection

In [8]:
[pprint(records) for records in finance_db.customers.find().limit(1)]

{'_id': ObjectId('5ca4bbcea2dd94ee58162a68'),
 'accounts': [371138, 324287, 276528, 332179, 422649, 387979],
 'active': True,
 'address': '9286 Bethany Glens\nVasqueztown, CO 22939',
 'birthdate': datetime.datetime(1977, 3, 2, 2, 20, 31),
 'email': 'arroyocolton@gmail.com',
 'name': 'Elizabeth Ray',
 'tier_and_details': {'0df078f33aa74a2e9696e0520c1a828a': {'active': True,
                                                           'benefits': ['sports '
                                                                        'tickets'],
                                                           'id': '0df078f33aa74a2e9696e0520c1a828a',
                                                           'tier': 'Bronze'},
                      '699456451cc24f028d2aa99d7534c219': {'active': True,
                                                           'benefits': ['24 '
                                                                        'hour '
                                              

[None]

### Display one record from transactions collection

In [9]:
[pprint(records) for records in finance_db.transactions.find().limit(1)]

{'_id': ObjectId('5ca4bbc1a2dd94ee58161cb1'),
 'account_id': 443178,
 'bucket_end_date': datetime.datetime(2017, 1, 3, 0, 0),
 'bucket_start_date': datetime.datetime(1969, 2, 4, 0, 0),
 'transaction_count': 66,
 'transactions': [{'amount': 7514,
                   'date': datetime.datetime(2003, 9, 9, 0, 0),
                   'price': '19.1072802650074180519368383102118968963623046875',
                   'symbol': 'adbe',
                   'total': '143572.1039112657392422534031',
                   'transaction_code': 'buy'},
                  {'amount': 9240,
                   'date': datetime.datetime(2016, 6, 14, 0, 0),
                   'price': '24.1525632387771480580340721644461154937744140625',
                   'symbol': 'team',
                   'total': '223169.6843263008480562348268',
                   'transaction_code': 'buy'},
                  {'amount': 2824,
                   'date': datetime.datetime(2002, 12, 4, 0, 0),
                   'price': '21.046193

[None]

### Get all the customer who has an active status and whose date of birth year is between 1995 and 1997

In [10]:
date_start = datetime(1995, 1, 1, 0, 0, 0, 0)
date_end = datetime(1997, 1, 1, 0, 0, 0, 0)

# Use the find() method to retrieve matching documents
matching_customers = customers.find({
    "birthdate": {
        "$gte": date_start,
        "$lte": date_end
    }
}).limit(5)

# Iterate and print the matching customers
for customer in matching_customers:
    pprint(customer)

{'_id': ObjectId('5ca4bbcea2dd94ee58162a6d'),
 'accounts': [904260, 565468],
 'address': '17677 Mark Crest\nWalterberg, IA 39017',
 'birthdate': datetime.datetime(1996, 9, 13, 17, 14, 27),
 'email': 'amyholland@yahoo.com',
 'name': 'Natalie Ford',
 'tier_and_details': {'07d516cfd7fc4ec6acf175bb78cb98a2': {'active': True,
                                                           'benefits': ['dedicated '
                                                                        'account '
                                                                        'representative'],
                                                           'id': '07d516cfd7fc4ec6acf175bb78cb98a2',
                                                           'tier': 'Gold'},
                      '69f8b6a3c39c42edb540499ee2651b75': {'active': True,
                                                           'benefits': ['dedicated '
                                                                        'account 

### Find all the unique products in the accounts collection
- Products is an array object in the accounts collection, there are multiple records, so we want to know how many unique products are there in this collection

In [11]:
pipeline = [
    
    {
        "$unwind": "$products"   #first unwind all the array into documents
    },
    {
        "$group": {
            "_id": "unique_products",
            "unique_products": {"$addToSet": "$products"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "unique_products": 1
        }
    }
]
accounts_list = list(accounts.aggregate(pipeline))
print(accounts_list[0]["unique_products"])

['Brokerage', 'CurrencyService', 'Commodity', 'InvestmentStock', 'Derivatives', 'InvestmentFund']


### Find all transactions which has purchased scrips for adobe, microsoft and netflix

In [12]:
pipeline = [
     {
        '$match': {
            'account_id': 443178
        }
    },
    {
        '$unwind': '$transactions'
    },
    {
        '$group': {
            '_id': {
                'account_id': '$account_id',
                'symbol': '$transactions.symbol'
            },
            'total': {
              '$sum': {
                    '$toDouble': '$transactions.total'
                }          
            }           
        }
    },   
    {
        '$project': {
            '_id': 0,
            'account_id': '$_id',
            'total': 1
        }
    }
]
transactions_list = list(transactions.aggregate(pipeline))
pprint(transactions_list)

[{'account_id': {'account_id': 443178, 'symbol': 'sap'},
  'total': 3606044.0271663205},
 {'account_id': {'account_id': 443178, 'symbol': 'team'},
  'total': 1114640.280341524},
 {'account_id': {'account_id': 443178, 'symbol': 'ibm'},
  'total': 4369375.981492583},
 {'account_id': {'account_id': 443178, 'symbol': 'nflx'},
  'total': 2752792.0209153784},
 {'account_id': {'account_id': 443178, 'symbol': 'msft'},
  'total': 1014534.0243606936},
 {'account_id': {'account_id': 443178, 'symbol': 'adbe'},
  'total': 1642509.9994569893}]


### Filter the customer who has a account and the limit is less then 1000, in this we will use a lookup to join the accounts and customer collection 
- we will lookup accounts with customers based on account_id

In [13]:
pipeline = [
    {
        "$match": {
            "limit": { "$lte": 5000 } # hard coded value , but will be replaced with variable when using with flask
        }
    },   
    {
        "$lookup": {
            "from": "customers",  
            "localField": "account_id",  
            "foreignField": "accounts",  
            "as": "customer"  # alias for the joined data
        }
    },
    {
        "$unwind": "$customer"  
    },       
    {
        "$project": {           
            "name": "$customer.name", 
            "email": "$customer.email", 
            "address": "$customer.address",              
            "limit": 1,                     
             "_id":0,
        }
    },
    {
        "$sort": {
            "limit": 1  # ascending order
        }
    }
]

result = accounts.aggregate(pipeline)

# Iterate through the results
for document in result:
    pprint(document)

{'address': '153 Ramirez Mountains\nHardyfort, IA 16827',
 'email': 'shane95@yahoo.com',
 'limit': 3000,
 'name': 'Colleen Watson'}
{'address': '22879 Peterson Hill\nEast Maria, NJ 24417',
 'email': 'robertlloyd@hotmail.com',
 'limit': 3000,
 'name': 'Danielle Blevins'}
{'address': '42601 Marc Forks\nLake Stevenbury, NE 09808',
 'email': 'vchristensen@gmail.com',
 'limit': 5000,
 'name': 'Daniel Wang'}


### To Display on the front end, we will join three collections and show selected fields from all the three collections
#### We wil use the following fields
- customers
- accounts
- transactions

In [14]:
pipeline = [
    {
        "$lookup": {
            "from": "customers",
            "localField": "account_id",
            "foreignField": "accounts",
            "as": "customer"
        }
    },
    {
        "$unwind": "$customer"
    },
    {
        "$match": {
            "customer.name": "Colleen Watson",             
        }
    },
    {
        "$lookup": {
            "from": "transactions",
            "localField": "account_id",
            "foreignField": "account_id",
            "as": "transactions"
        }
    },
    {
        "$unwind" : "$transactions"
    },
    {
        "$project": {
            "_id": 0,
            "customerName": "$customer.name",
            "accountLimit": "$limit",   
            "bucket_end_date" : "$transactions.bucket_end_date",
            "bucket_start_date" : "$transactions.bucket_start_date",
            "transaction_count" : "$transactions.transaction_count"
        }
    }
]

result = accounts.aggregate(pipeline)

for document in result:
    print(document)


{'customerName': 'Colleen Watson', 'accountLimit': 10000, 'bucket_end_date': datetime.datetime(2017, 1, 4, 0, 0), 'bucket_start_date': datetime.datetime(1989, 1, 7, 0, 0), 'transaction_count': 45}
{'customerName': 'Colleen Watson', 'accountLimit': 10000, 'bucket_end_date': datetime.datetime(2017, 1, 9, 0, 0), 'bucket_start_date': datetime.datetime(1987, 9, 12, 0, 0), 'transaction_count': 99}
{'customerName': 'Colleen Watson', 'accountLimit': 10000, 'bucket_end_date': datetime.datetime(2016, 12, 31, 0, 0), 'bucket_start_date': datetime.datetime(1986, 5, 21, 0, 0), 'transaction_count': 89}
{'customerName': 'Colleen Watson', 'accountLimit': 3000, 'bucket_end_date': datetime.datetime(2007, 12, 19, 0, 0), 'bucket_start_date': datetime.datetime(2006, 4, 28, 0, 0), 'transaction_count': 1}
{'customerName': 'Colleen Watson', 'accountLimit': 10000, 'bucket_end_date': datetime.datetime(2017, 1, 3, 0, 0), 'bucket_start_date': datetime.datetime(1987, 11, 16, 0, 0), 'transaction_count': 82}
{'custom

### CRUD operation to insert, update and delete new records into Customers collection

In [15]:
new_customer = {
         'accounts': [371138],
         'active': True,
         'address': 'ABC India Town, Bangalore, PIN 458585',
         'birthdate': datetime(1977, 3, 2, 2, 20, 31),
         'email': 'test@gmail.com',
         'name': 'Abby Roy',
         'tier_and_details': {'0df078f33aa74a2e9696e0520c1a828b': {'active': True,
                                                                   'benefits': ['sports '
                                                                                'tickets'],
                                                                   'id': '0dg078f33aa74a2e9696e0520c1a828a',
                                                                   'tier': 'Bronze'
                                                                  }
                              },
         'username': 'aroy'
    }
    
result = customers.insert_one(new_customer)
print(f"Generated Id for Inserted record is : {result.inserted_id}")

Generated Id for Inserted record is : 6512649f80332d8e49e479cc


### CRUD operation to update the customer collection for single record

In [16]:
filter_criteria = {"email": "test@gmail.com"} 

update_data = {"$set": {"name": "BBC Roy", "active": False}}

result = customers.update_one(filter_criteria, update_data)
print(result.modified_count)

search_result = customers.find({"email":"test@gmail.com"})

for document in search_result:
    print(document)

1
{'_id': ObjectId('6512649f80332d8e49e479cc'), 'accounts': [371138], 'active': False, 'address': 'ABC India Town, Bangalore, PIN 458585', 'birthdate': datetime.datetime(1977, 3, 2, 2, 20, 31), 'email': 'test@gmail.com', 'name': 'BBC Roy', 'tier_and_details': {'0df078f33aa74a2e9696e0520c1a828b': {'active': True, 'benefits': ['sports tickets'], 'id': '0dg078f33aa74a2e9696e0520c1a828a', 'tier': 'Bronze'}}, 'username': 'aroy'}


In [17]:
result = customers.delete_one({"email":"test@gmail.com"})
print(result.deleted_count)

1


### Filter by limit field in accounts using greater then clause

In [18]:
pipeline = [
   {
      "$lookup":{
         "from":"customers",
         "localField":"account_id",
         "foreignField":"accounts",
         "as":"customer"
      }
   },
   {
      "$unwind":"$customer"
   },
   {
      "$lookup":{
         "from":"transactions",
         "localField":"account_id",
         "foreignField":"account_id",
         "as":"transactions"
      }
   },
   {
      "$unwind":"$transactions"
   },   
   {
      "$match":{
         "limit":{
            "$gte": 1000
         }
      }
   },
   {
      "$limit":5
   },
   {
      "$project":{
         "_id":0,
         "name":"$customer.name",
         "email":"$customer.email",
         "userName":"$customer.username",
         "accountLimit":"$limit",
         "transactionCount":"$transactions.transaction_count"
      }
   }
]

result = accounts.aggregate(pipeline)

for document in result:
    print(document)

{'name': 'Colleen Watson', 'email': 'shane95@yahoo.com', 'userName': 'martinallen', 'accountLimit': 3000, 'transactionCount': 1}
{'name': 'Danielle Blevins', 'email': 'robertlloyd@hotmail.com', 'userName': 'tina17', 'accountLimit': 3000, 'transactionCount': 2}
{'name': 'Daniel Wang', 'email': 'vchristensen@gmail.com', 'userName': 'dianefoster', 'accountLimit': 5000, 'transactionCount': 1}
{'name': 'Wendy Lane', 'email': 'kristinmartinez@hotmail.com', 'userName': 'garymiller', 'accountLimit': 7000, 'transactionCount': 3}
{'name': 'Robert Hooper', 'email': 'stephenmassey@hotmail.com', 'userName': 'nicolesmith', 'accountLimit': 7000, 'transactionCount': 1}


### Search by username using find command and project the required fields

In [19]:
result = customers.find(
        {"username": 'fmiller'},
        {"_id": 0, "name": 1, "email": 1, "username": 1, "birthdate": 1})
for document in result:
    pprint(document)

{'birthdate': datetime.datetime(1977, 3, 2, 2, 20, 31),
 'email': 'arroyocolton@gmail.com',
 'name': 'Elizabeth Ray',
 'username': 'fmiller'}
